In [ ]:
import os
import json
import base64
import pandas as pd
from users import get_user_id
from requests import post, get
from dotenv import load_dotenv
from utils import ms_to_min_sec
from time import sleep
from validate import get_token, get_auth_header

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
token = get_token()

In [ ]:
def search_for_artist(token, artist_name):
    url = 'https://api.spotify.com/v1/search'
    headers = get_auth_header(token)
    query = f'q={artist_name}&type=artist&limit=1'
    query_url = url + '?' + query

    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)[ 'artists']['items']
    return json_result


In [ ]:
def search_playlist_items(token,  playlist_id, total_songs):
    headers = get_auth_header(token)

    offset=0
    song_amnt = total_songs
    tracks = []
    while song_amnt > 0:
        url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?limit=100&offset={offset}'
        result = get(url, headers=headers)
        json_result = json.loads(result.content)['items']

        for item in json_result:
            track = {}
            track['name'] = item["track"]["name"]
            track['artist'] = item["track"]["artists"][0]["name"]
            track['album'] = item["track"]["album"]["name"]
            track['release_date'] = item["track"]["album"]["release_date"]
            track['duration_ms'] = item["track"]["duration_ms"]
            track['duration'] = ms_to_min_sec(item["track"]["duration_ms"])
            track['url'] = item["track"]["href"]
            track['track_id'] = item['track']['id']
            tracks.append(track)

        song_amnt -= 100
        offset +=100
    return pd.DataFrame(tracks)

In [ ]:
def get_general_tracks(tracks_ids):
    infos = []
    headers = get_auth_header(token)
    for track_id in tracks_ids:
        url = f'https://api.spotify.com/v1/tracks/{track_id}'
        general_result = get(url, headers=headers)
        json_result = json.loads(general_result.content)
        print(json_result)
        info = {}
        info['track_name'] = json_result['name']
        info['album_name'] = json_result['album']['name']
        info['artist_name'] = json_result['artists'][0]['name']
        info['image'] = json_result['album']['images'][0]
        info['release_date'] = json_result['album']['release_date']
        info['explicit'] = json_result['explicit']
        info['popularity'] = json_result['popularity']
        infos.append(info)
    return pd.DataFrame(infos)

In [ ]:
def get_feature_tracks(tracks_ids):
    infos = []
    headers = get_auth_header(token)
    for track_id in tracks_ids:
        url = f'https://api.spotify.com/v1/audio-features/{track_id}'
        general_result = get(url, headers=headers)
        json_result = json.loads(general_result.content)
        print(json_result)

        info = {}
        info['danceability'] = json_result['danceability']
        info['energy'] = json_result['energy']
        info['key'] = json_result['key']
        info['loudness'] = json_result['loudness']
        info['mode'] = json_result['mode']
        info['speechiness'] = json_result['speechiness']
        info['acousticness'] = json_result['acousticness']
        info['instrumentalness'] = json_result['instrumentalness']
        info['liveness'] = json_result['liveness']
        info['valence'] = json_result['valence']
        info['tempo'] = json_result['tempo']
        infos.append(info)
    return pd.DataFrame(infos)

In [ ]:
def get_tracks_info(token, tracks_ids):
    headers = get_auth_header(token)
    
    ids = '%2C'.join(tracks_ids.astype(str))

    url1 = f'https://api.spotify.com/v1/tracks?ids={ids}'
    general_result = get(url1, headers=headers)
    json_result_general = json.loads(general_result.content)

    url2 = f'https://api.spotify.com/v1/audio-features?ids={ids}'
    features_result = get(url2, headers=headers)
    json_result_features = json.loads(features_result.content)
    print(json_result_general)
    print(json_result_features)

    json_result_general = json_result_general['tracks']
    infos_general = []
    for item in json_result_general:
        info = {}
        info['track_name'] = item['name']
        info['album_name'] = item['album']['name']
        info['artist_name'] = item['artists'][0]['name']
        info['image'] = item['album']['images'][0]
        info['release_date'] = item['album']['release_date']
        info['explicit'] = item['explicit']
        info['popularity'] = item['popularity']
        infos_general.append(info)


    json_result_features = json_result_features['audio_features']
    infos_features = []
    for item in json_result_features:
        info = {}
        info['danceability'] = item['danceability']
        info['energy'] = item['energy']
        info['key'] = item['key']
        info['loudness'] = item['loudness']
        info['mode'] = item['mode']
        info['speechiness'] = item['speechiness']
        info['acousticness'] = item['acousticness']
        info['instrumentalness'] = item['instrumentalness']
        info['liveness'] = item['liveness']
        info['valence'] = item['valence']
        info['tempo'] = item['tempo']
        infos_features.append(info)

    df = pd.concat([pd.DataFrame(infos_general), pd.DataFrame(infos_features)], axis=1)
    return df


In [ ]:
def get_playlist_names(token, user_id):
    url =f'https://api.spotify.com/v1/users/{user_id}/playlists'
    headers = get_auth_header(token)

    result = get(url, headers=headers)
    json_result = json.loads(result.content)['items']

    playlists = []
    for item in json_result:
        playlist = {}
        playlist['id'] = item['id']
        playlist['playlist_name'] = item['name']
        playlist['owner_name'] = item['owner']['display_name']
        playlist['url'] = item['tracks']['href']
        playlist['total_songs'] = item['tracks']['total']
        playlists.append(playlist)
    return pd.DataFrame(playlists)


In [ ]:
user_playlists = get_playlist_names(token, get_user_id()[1]).reset_index()
user_playlists_over_100=user_playlists[user_playlists['total_songs']>=100]
user_playlists_under_100=user_playlists[user_playlists['total_songs']<100]

In [ ]:
user_playlists_over_100

In [ ]:
user_playlists_under_100

In [ ]:
dfs = []

In [ ]:
# for id in user_playlists_over_100.index:
#     print(id)
#     playlist_items = search_playlist_items(token,  playlist_id=user_playlists.id.loc[id], total_songs=user_playlists.total_songs.loc[id])
#     df1 = get_general_tracks(playlist_items.track_id)
#     df2 = get_feature_tracks(playlist_items.track_id)
#     df = pd.concat([df1, df2], axis=1)
#     dfs.append(df)
#     sleep(3)

In [ ]:
for id in user_playlists_under_100.index:
    print(id)
    playlist_items = search_playlist_items(token,  playlist_id=user_playlists_under_100.id.loc[id], total_songs=user_playlists.total_songs.loc[id])
    df = get_tracks_info(token, playlist_items.track_id)
    dfs.append(df)
    sleep(3)

In [ ]:
df_final = pd.concat(dfs)

In [ ]:
df_final.shape

In [ ]:
user = get_user_id()[1]

In [ ]:
def get_user_final_whole_playlists(user):
    user_playlists = get_playlist_names(token, user).reset_index()
    user_playlists_over_100=user_playlists[user_playlists['total_songs']>=100]
    user_playlists_under_100=user_playlists[user_playlists['total_songs']<100]

    dfs = []

    for id in user_playlists_under_100.index:
        print(id)
        playlist_items = search_playlist_items(token,  playlist_id=user_playlists_under_100.id.loc[id], total_songs=user_playlists.total_songs.loc[id])
        df = get_tracks_info(token, playlist_items.track_id)
        dfs.append(df)
        sleep(3)

    for id in user_playlists_over_100.index:
        print(id)
        playlist_items = search_playlist_items(token,  playlist_id=user_playlists.id.loc[id], total_songs=user_playlists.total_songs.loc[id])
        df1 = get_general_tracks(playlist_items.track_id)
        df2 = get_feature_tracks(playlist_items.track_id)
        df = pd.concat([df1, df2], axis=1)
        dfs.append(df)
        sleep(3)
    
    df_final = pd.concat(dfs)

    return df_final


In [ ]:
get_user_final_whole_playlists(user)